# Product Ranking for Upselling

## Objective
Create a ranking system for product recommendations based on **Expected Value**:

$$\text{Expected Value} = P(\text{purchase}) \times \text{Price}$$

This helps identify which products to recommend for maximum revenue potential.

### Use Cases:
1. **General Upsell**: What products to recommend to any customer
2. **Contextual Upsell**: What to recommend based on current cart
3. **Cross-sell Opportunities**: Products frequently bought together

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from data_loader import load_raw_data, SAUCES

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

## 1. Load and Prepare Data

In [ ]:
# Load data
DATA_PATH = Path('../data/raw/ap_dataset.csv')
df = load_raw_data(DATA_PATH)

print(f"Dataset: {len(df)} rows, {df['id_bon'].nunique()} receipts")

total_receipts = df['id_bon'].nunique()

In [ ]:
# Calculate product statistics
product_stats = df.groupby('retail_product_name').agg({
    'id_bon': 'nunique',  # Number of receipts containing product
    'SalePriceWithVAT': 'first'  # Price (assuming constant)
}).reset_index()

product_stats.columns = ['product', 'receipt_count', 'price']
product_stats['purchase_probability'] = product_stats['receipt_count'] / total_receipts
product_stats['expected_value'] = product_stats['purchase_probability'] * product_stats['price']

# Sort by expected value
product_stats = product_stats.sort_values('expected_value', ascending=False)

print("Top 10 Products by Expected Value:")
product_stats.head(10)

## 2. General Upsell Ranking

In [ ]:
# Visualize top 20 products by expected value
top_20 = product_stats.head(20)

fig, ax = plt.subplots(figsize=(12, 10))
colors = plt.cm.viridis(np.linspace(0, 1, len(top_20)))

bars = ax.barh(range(len(top_20)), top_20['expected_value'], color=colors)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['product'])
ax.set_xlabel('Expected Value (RON)')
ax.set_title('Top 20 Products by Expected Value for Upselling')
ax.invert_yaxis()

# Add value labels
for i, (v, p) in enumerate(zip(top_20['expected_value'], top_20['purchase_probability'])):
    ax.text(v + 0.1, i, f'{v:.2f} (P={p:.1%})', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../results/ranking_expected_value.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter plot: Price vs Probability with Expected Value as size
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(
    product_stats['price'],
    product_stats['purchase_probability'],
    s=product_stats['expected_value'] * 50,  # Size proportional to EV
    c=product_stats['expected_value'],
    cmap='YlOrRd',
    alpha=0.7,
    edgecolors='black',
    linewidth=0.5
)

plt.colorbar(scatter, label='Expected Value')

# Annotate top products
for _, row in product_stats.head(5).iterrows():
    ax.annotate(row['product'], (row['price'], row['purchase_probability']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

ax.set_xlabel('Price (RON)')
ax.set_ylabel('Purchase Probability')
ax.set_title('Product Price vs Purchase Probability\n(Size = Expected Value)')
plt.tight_layout()
plt.savefig('../results/ranking_price_vs_probability.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Category-Based Ranking

In [ ]:
# Categorize products
def categorize_product(name):
    name_lower = name.lower()
    if 'sauce' in name_lower:
        return 'Sauce'
    elif 'schnitzel' in name_lower:
        return 'Schnitzel'
    elif any(x in name_lower for x in ['fries', 'potatoes', 'baked']):
        return 'Sides'
    elif any(x in name_lower for x in ['pepsi', 'cola', 'aqua', '7up', 'lipton', 'mirinda', 'dew', 'prigat']):
        return 'Drinks'
    elif 'salad' in name_lower:
        return 'Salad'
    elif 'mac' in name_lower:
        return 'Mac & Cheese'
    else:
        return 'Other'

product_stats['category'] = product_stats['product'].apply(categorize_product)

# Category summary
category_stats = product_stats.groupby('category').agg({
    'expected_value': ['sum', 'mean', 'max'],
    'product': 'count'
})
category_stats.columns = ['total_ev', 'avg_ev', 'max_ev', 'product_count']
category_stats = category_stats.sort_values('total_ev', ascending=False)

print("Category Statistics:")
category_stats

In [ ]:
# Top product per category
print("\nTop Product by Expected Value in Each Category:")
print("=" * 70)

for category in category_stats.index:
    cat_products = product_stats[product_stats['category'] == category]
    top = cat_products.iloc[0]
    print(f"\n{category}:")
    print(f"  {top['product']}")
    print(f"  Price: {top['price']:.2f} RON")
    print(f"  Purchase Probability: {top['purchase_probability']:.1%}")
    print(f"  Expected Value: {top['expected_value']:.2f} RON")

In [ ]:
# Visualization by category
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

categories = ['Schnitzel', 'Sides', 'Sauce', 'Drinks', 'Salad', 'Mac & Cheese']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD']

for ax, category, color in zip(axes, categories, colors):
    cat_data = product_stats[product_stats['category'] == category].head(5)
    
    if len(cat_data) > 0:
        ax.barh(range(len(cat_data)), cat_data['expected_value'], color=color)
        ax.set_yticks(range(len(cat_data)))
        ax.set_yticklabels([p[:25] + '...' if len(p) > 25 else p for p in cat_data['product']])
        ax.set_xlabel('Expected Value (RON)')
        ax.set_title(f'{category}')
        ax.invert_yaxis()
    else:
        ax.text(0.5, 0.5, 'No products', ha='center', va='center')
        ax.set_title(f'{category}')

plt.suptitle('Top 5 Products by Expected Value per Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/ranking_by_category.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Contextual Ranking (Based on Cart)

In [ ]:
def calculate_conditional_ev(df, given_product, total_receipts):
    """
    Calculate expected value of products given that a specific product is in the cart.
    
    P(Y | X) = P(X and Y) / P(X)
    
    Args:
        df: DataFrame with transactions
        given_product: The product already in cart
        total_receipts: Total number of receipts
        
    Returns:
        DataFrame with conditional expected values
    """
    # Receipts containing the given product
    given_receipts = df[df['retail_product_name'] == given_product]['id_bon'].unique()
    n_given = len(given_receipts)
    
    if n_given == 0:
        return pd.DataFrame()
    
    # Filter to those receipts
    filtered_df = df[df['id_bon'].isin(given_receipts)]
    
    # Calculate stats for other products
    results = []
    for product in df['retail_product_name'].unique():
        if product == given_product:
            continue
            
        # Receipts with both products
        with_product = filtered_df[filtered_df['retail_product_name'] == product]['id_bon'].nunique()
        
        # Conditional probability
        cond_prob = with_product / n_given
        
        # Get price
        price = df[df['retail_product_name'] == product]['SalePriceWithVAT'].iloc[0]
        
        # Expected value
        expected_value = cond_prob * price
        
        results.append({
            'product': product,
            'conditional_probability': cond_prob,
            'price': price,
            'conditional_expected_value': expected_value
        })
    
    result_df = pd.DataFrame(results)
    return result_df.sort_values('conditional_expected_value', ascending=False)

In [ ]:
# Example: What to recommend when customer has Crazy Schnitzel
cs_recommendations = calculate_conditional_ev(df, 'Crazy Schnitzel', total_receipts)

print("Top 10 Upsell Recommendations for Crazy Schnitzel buyers:")
print("=" * 70)
cs_recommendations.head(10)

In [ ]:
# Visualize recommendations for Crazy Schnitzel
top_10 = cs_recommendations.head(10)

fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.Oranges(np.linspace(0.3, 1, len(top_10)))

bars = ax.barh(range(len(top_10)), top_10['conditional_expected_value'], color=colors)
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels(top_10['product'])
ax.set_xlabel('Expected Value (RON)')
ax.set_title('Top 10 Upsell Recommendations for Crazy Schnitzel Buyers\n(Conditional Expected Value)')
ax.invert_yaxis()

# Add value labels
for i, (v, p) in enumerate(zip(top_10['conditional_expected_value'], top_10['conditional_probability'])):
    ax.text(v + 0.05, i, f'{v:.2f} (P={p:.1%})', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../results/ranking_crazy_schnitzel_upsell.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Compare recommendations for different main dishes
main_dishes = [
    'Crazy Schnitzel',
    'Breaded Chicken Schnitzel',
    'Crispy Chicken Schnitzel',
    'Viennese Schnitzel'
]

comparison = {}
for dish in main_dishes:
    recs = calculate_conditional_ev(df, dish, total_receipts)
    if len(recs) > 0:
        comparison[dish] = recs.head(5)[['product', 'conditional_expected_value']].to_dict('records')

print("Top 5 Upsell Recommendations by Main Dish:")
print("=" * 70)
for dish, recs in comparison.items():
    print(f"\n{dish}:")
    for i, rec in enumerate(recs, 1):
        print(f"  {i}. {rec['product']}: {rec['conditional_expected_value']:.2f} RON")

## 5. Sauce-Specific Ranking

In [ ]:
# Ranking for sauces only
sauce_stats = product_stats[product_stats['category'] == 'Sauce'].copy()

print("Sauce Ranking by Expected Value:")
print("=" * 60)
for i, row in sauce_stats.iterrows():
    print(f"{row['product']:25s} EV: {row['expected_value']:.2f} RON (P={row['purchase_probability']:.1%})")

In [ ]:
# Sauce Expected Value visualization
fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.Reds(np.linspace(0.3, 1, len(sauce_stats)))
bars = ax.bar(range(len(sauce_stats)), sauce_stats['expected_value'], color=colors)
ax.set_xticks(range(len(sauce_stats)))
ax.set_xticklabels(sauce_stats['product'], rotation=45, ha='right')
ax.set_ylabel('Expected Value (RON)')
ax.set_title('Sauce Ranking by Expected Value')

# Add probability labels
for i, (v, p) in enumerate(zip(sauce_stats['expected_value'], sauce_stats['purchase_probability'])):
    ax.text(i, v + 0.02, f'P={p:.1%}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('../results/ranking_sauce_ev.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Create Ranking Function

In [ ]:
class UpsellRanker:
    """
    Product ranking system for upselling based on expected value.
    """
    
    def __init__(self, df):
        self.df = df
        self.total_receipts = df['id_bon'].nunique()
        
        # Pre-compute product stats
        self.product_stats = self._compute_product_stats()
    
    def _compute_product_stats(self):
        stats = self.df.groupby('retail_product_name').agg({
            'id_bon': 'nunique',
            'SalePriceWithVAT': 'first'
        }).reset_index()
        stats.columns = ['product', 'receipt_count', 'price']
        stats['probability'] = stats['receipt_count'] / self.total_receipts
        stats['expected_value'] = stats['probability'] * stats['price']
        return stats
    
    def get_general_ranking(self, top_n=10, exclude_products=None):
        """
        Get top products to recommend in general.
        """
        ranking = self.product_stats.copy()
        if exclude_products:
            ranking = ranking[~ranking['product'].isin(exclude_products)]
        return ranking.nlargest(top_n, 'expected_value')
    
    def get_conditional_ranking(self, given_product, top_n=10, exclude_categories=None):
        """
        Get top products to recommend given a product in cart.
        """
        # Get receipts with given product
        given_receipts = self.df[self.df['retail_product_name'] == given_product]['id_bon'].unique()
        n_given = len(given_receipts)
        
        if n_given == 0:
            return pd.DataFrame()
        
        filtered_df = self.df[self.df['id_bon'].isin(given_receipts)]
        
        results = []
        for product in self.df['retail_product_name'].unique():
            if product == given_product:
                continue
            
            with_product = filtered_df[filtered_df['retail_product_name'] == product]['id_bon'].nunique()
            cond_prob = with_product / n_given
            price = self.df[self.df['retail_product_name'] == product]['SalePriceWithVAT'].iloc[0]
            
            results.append({
                'product': product,
                'conditional_probability': cond_prob,
                'price': price,
                'expected_value': cond_prob * price
            })
        
        result_df = pd.DataFrame(results)
        return result_df.nlargest(top_n, 'expected_value')

# Create ranker instance
ranker = UpsellRanker(df)

# Test
print("General Top 5 Recommendations:")
print(ranker.get_general_ranking(top_n=5))

print("\nConditional Recommendations (given French fries):")
print(ranker.get_conditional_ranking('French fries', top_n=5))

## 7. Summary

In [ ]:
print("=" * 70)
print("PRODUCT RANKING SYSTEM SUMMARY")
print("=" * 70)

print(f"\nDataset: {len(df):,} transactions, {total_receipts:,} receipts")
print(f"Products analyzed: {len(product_stats)}")

print("\n" + "─" * 70)
print("GENERAL UPSELL RANKING (Top 5):")
print("─" * 70)
for i, row in product_stats.head(5).iterrows():
    print(f"  {row['product']:35s} EV: {row['expected_value']:.2f} RON")

print("\n" + "─" * 70)
print("BEST SAUCE TO RECOMMEND:")
print("─" * 70)
best_sauce = sauce_stats.iloc[0]
print(f"  {best_sauce['product']}")
print(f"  Expected Value: {best_sauce['expected_value']:.2f} RON")
print(f"  Purchase Probability: {best_sauce['purchase_probability']:.1%}")

print("\n" + "─" * 70)
print("KEY INSIGHT:")
print("─" * 70)
print("  Use conditional expected value for personalized recommendations.")
print("  The UpsellRanker class provides both general and contextual rankings.")
print("=" * 70)